# KNN exercise

##### (Notebook By Itay Hazan)

In [4]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.colors import ListedColormap
from sklearn import neighbors

## Step 1: Visualizing KNN

Complete the following code that samples $n=30$ uniformly-distributed random points in $ [0,1] \times [0,1] $ and samples a label between 0 and `max_label`=3, that is sample $X \in \left([0,1]\times [0,1]\right)^{n \times 2} $ and $ y \in \{0,\dots, \text{max_label} \}^{n \times 1} $:

In [2]:
def generate_points_with_labels(n=30, max_label=3):
    # TODO
    return X, y

Call the function above and verify the shapes of $X$ and $y$:

In [ ]:
X, y = generate_points_with_labels()
print(X.shape, y.shape)

Plot the points, and color each label according to its label.

Create an instance of `sklearn`'s KNN, with $k=1$.

In [3]:
k = 1
classifier = None

Fit the classifier to your data. Use uniform weights.

Visualize the classification regions of $[0,1]\times [0,1]$. You might find [this](https://scikit-learn.org/stable/auto_examples/neighbors/plot_classification.html#sphx-glr-auto-examples-neighbors-plot-classification-py) code example useful.

Repeat this step for different values of $k$, and make sure you understand how the classification regions change.

## Step 2: Choosing an Optimal Value for k Using Cross Validation

In this part of the exercise, we'll use cross validation to choose the best value for our hyperparameter, $k$.

First, we'll have to make our label-generation a bit more sophisticated than just plain random numbers (why?).

We begin by sampling our points in $[-1,1]\times [-1,1]$. Now, we will attempt to sample the labels such that points in the first and third quadrant will have a higher probability of samplying a 1-label ($\Pr [label = 1] = \frac{3}{4}$), and points in the second and fourth quadrant will have a higher probability of sampling a 0-label ($\Pr [label = 0] = \frac{3}{4}$). 

This can be done quite easily by defining the following the label of each sample `X[i,:]=`$(x_1, x_2)$ as a Bernoulli random variable with probability of success $$ p = \frac{1}{2} + (-1)^{\mathrm{sign} (x_1) \cdot \mathrm{sign} (x_2)} \cdot \frac{1}{4}. $$

In [5]:
def generate_points_with_labels(n=100):
    X, _ = generate_points_with_labels(n=n, max_label=1) # only two kinds of labels, 0 or 1
    X = X*2-1 # transforms the points in X to [-1,1]X[-1,1] 
    
    y = None # TODO: numpy.random.binomial might be useful
    return X, y

Plot the points, and color each label according to its label.

In [6]:
# TODO: plot the points

Split the set of points into two sets, `train` and `test`, such that `train` contains $80\%$ of the data and `test` contains the other $20\%$.

In [ ]:
train_X, train_y = 
test_X, test_y = 

Split the train test into 4-folds (equal-size parts, i.e. $20\%$ of the original data each).

In [ ]:
folds_Xs, folds_ys = 

Find the best value for $k$ using 4-fold cross validation, that is:

1. For every value of $k$ from 1 to 10,
 1. For every $i$ from 0 to 3,
     1. Create KNN classifier with current $k$.
     1. Fit the classifier on $ \text{train} \setminus \text{fold}_i $.
     1. Evaluate the accuracy of classifier on $\text{fold}_i$.
        
Save all evaluations (throughout all iterations) in a dictionary of lists ` { k => [list of four accuracies] }`.

In [8]:
# TODO: implement 4-fold cross validation

For every $k$, compute its mean accuracy (taken on all folds). Save the results a $ (\text{num_values_of_k}) \times 2 $ array, where the $i$th row contains is `[k_i, mean_accuracy_i]`.

In [9]:
# TODO: compute mean accuracies

Plot a graph of mean accuracies.

In [10]:
# TODO

Conclude the optimal value of $k$ given your train set.

In [11]:
k_opt = 

Create a KNN classifier with $k_{opt}$, and evaluate its accuracy on the test set.

In [ ]:
# TODO